In [1]:
%pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import tensorflow as tf
import keras
import numpy as np

from keras.src.layers import Embedding
from keras.layers import Dense
from keras.src.utils import to_categorical
from keras.models import Sequential

from scipy.sparse import csr_matrix

from numpy.lib.stride_tricks import sliding_window_view


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.utils import compute_sample_weight


/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [25]:
df_r4 = pd.read_csv('../data_mod/R4_Train_poslabel.csv')
df_r4_interleaved = pd.read_csv('../data_mod/R4_HR_Interleaved_poslabel.csv')

df_r1 = pd.read_csv('../data_mod/R1_Train_poslabel.csv')
df_r1_interleaved = pd.read_csv('../data_mod/R1_HR_Interleaved_poslabel.csv')

df2_r4 = pd.read_csv('../data_mod/R4/R4_Valid.csv')

In [27]:
df_r4["event_with_roles"].value_counts()

event_with_roles
Odoo Application->End Point (HR Manager): [HttpResponse:HTTP/1.0 200 OK\r\n]                                               4725
End Point (HR Manager)->Odoo Application: [HttpRequest:POST /xmlrpc/2/object HTTP/1.1\r\n]                                 3032
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['hr_applicant']]                               2233
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['mail_message_res_partner_needaction_rel']]    2067
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['mail_message']]                               2067
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:INSERT:['mail_message']]                               1908
End Point (HR Manager)->Odoo Application: [HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n]                                 1685
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['mail_mail'

In [29]:
def rindex(lst, value, start, end):
    start_rev = len(lst)-end
    end_rev = len(lst)-start
    r = len(lst) - 1 - lst[::-1].index(value,start_rev,end_rev)
    return r

def evaluate_interleaved(pred, y, labels):
    start_window = 27
    end_window = 51
    

    # transform input to integer labels
    pred_int = np.argmax(pred, axis = 1).tolist()
    y_int = np.argmax(y, axis = 1).tolist()
    
    in_between = labels.index("position_between")
    end = labels.index("position_end")
    start = labels.index("position_start")
    
    # For predicted R1 start events, the R4 label should be in a forward facing window of size 27
    # For predicted R1 end events the R4 label should be in a backward facing window of size 51
    # Since we introduce a window for the R4 labels, the original labels should be changed to "in between". 
    
    # get classification results for Start Events 
    
    # Idea: Reorder the labels greedy based on the window sizes! 
    # For start events in pred: Find swap candidate in y_int sublist index + window size, swap "forward"
    # For end events in results: Find swap candidate in pred index + size, swap "backward"
    # Todo Investigate: Only allow swaps with "Between elements"
        
    for idx in range(len(pred)):

        if(pred_int[idx] == start):
            try:# start event in predictions found -> check for start event in labels with forward facing window "forward swapping"            
                res_idx = y_int.index(start, idx,idx+start_window)
                if(y_int[idx] == in_between):
                    y_int[idx], y_int[res_idx] = y_int[res_idx], y_int[idx]
            except:
                pass

        bw_idx = len(pred)-idx-1
        if(pred_int[bw_idx]==end):
            try: 
                # Todo Gets Leftmost index currently - change to rightmost
                res_idx = rindex(y_int, end, max(bw_idx-end_window, 0), bw_idx)
                if y_int[bw_idx] == in_between :
                    y_int[bw_idx], y_int[res_idx] = y_int[res_idx], y_int[bw_idx]
            except: 
                pass
    
    report = classification_report(y_int, pred_int, target_names = labels)
    confusion_matrix = multilabel_confusion_matrix(y_int, pred_int)
    print(report)
    print(confusion_matrix)

In [54]:
from keras.src.layers import LSTM
from keras.layers import Bidirectional
from keras import Input
from keras.metrics import F1Score

def baseline_model(sequence_length, embedding_vocab_size):
    embedding_dim = 150


    model = Sequential([
        Input(shape=(sequence_length,)),
        Embedding(input_dim=embedding_vocab_size, output_dim=embedding_dim),
        LSTM(units=64),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[F1Score()])
    return model


In [63]:
    
def sliding_window(observation, forward, backward): 
    window_size = forward+backward+1
    
    sequences = sliding_window_view(observation, window_size)
    
    # pad first n-1 elements
    #pad_top = [[0] * (sequence_length - 1 - i) + sequences[0, 0:i + 1].tolist() for i in range(sequence_length - 1)]
    
    #res = np.insert(sequences,0, pad_top, axis=0)
    #res = np.asarray(X).astype(np.float32) 
    #res = res.reshape(res.shape[0], res.shape[1], 1)
    return sequences

In [74]:
labels = [x for x in range(100)]
window = sliding_window([x for x  in range(100)], 5,10)

In [86]:
# encode message type for both training and validation data
message_type_encoder = LabelEncoder().fit(df_r1["MessageType"].values)
train_features = message_type_encoder.transform(df_r1["MessageType"]) +1

# generate feature sequences 
train_feature_sequence = sliding_window(train_features,10, 10)

In [6]:
def mark_event_positions(df):
    # Mark start event of each BusinessActivity Instance
    df["activityStart"] = df.groupby(["BusinessActivity","InstanceNumber",]).cumcount()==0
    # Mark end event of each Business Activity Instance
    df["activityEnd"] = df.groupby(["BusinessActivity","InstanceNumber",]).cumcount(ascending=False)==0
    # Merge start and end columns to form labels
    df["task_position"] = df.apply(lambda row: "position_start" if row["activityStart"] else ("position_end" if row["activityEnd"] else 'position_between'), axis=1)

    df = df.drop(["activityStart",'activityEnd'], axis=1)
    
def sliding_window(observation, sequence_length = 50): 
    sequences = sliding_window_view(observation, sequence_length)
    
    # pad first n-1 elements
    pad_top = [[0] * (sequence_length - 1 - i) + sequences[0, 0:i + 1].tolist() for i in range(sequence_length - 1)]
    
    res = np.insert(sequences,0, pad_top, axis=0)
    #res = np.asarray(X).astype(np.float32) 
    #res = res.reshape(res.shape[0], res.shape[1], 1)
    return res

def evaluate( pred,y_test, labels):
    report = classification_report(y_test, pred, target_names = labels)
    confusion_matrix = multilabel_confusion_matrix(y_test, pred)
    print(report)
    print(confusion_matrix)
    return report, confusion_matrix
    
    

def pipeline(df_train, df_valid, model_builder, sequence_length, epochs, batch_size, weight_samples = False):
    
    
    # relabel task position for interleaved data
    # df_valid["task_position"] = df_valid.apply(lambda row: "position_start" if row["real_single_activity_action"] == "Activity Start" else ("position_end" if row["real_single_activity_action"] == "Activity End" else "position_between"), axis = 1)

    # mark event positions in training data
    # mark_event_positions(df_train)

    # encode message type for both training and validation data
    message_type_encoder = LabelEncoder().fit(df_train["MessageType"].values)
    train_features = message_type_encoder.transform(df_train["MessageType"]) +1
    valid_features = message_type_encoder.transform(df_valid["MessageType"]) +1 

    # generate feature sequences 
    train_feature_sequence = sliding_window(train_features,sequence_length)
    valid_feature_sequence = sliding_window(valid_features, sequence_length)

    # one hot encode labels
    label_encoder = LabelEncoder().fit(df_train["task_position"])
    train_labels = to_categorical(label_encoder.transform(df_train["task_position"]))
    valid_labels = to_categorical(label_encoder.transform(df_valid["task_position"]))

    # train test split
    x_train, x_test, y_train, y_test = train_test_split(train_feature_sequence, train_labels, test_size = 0.2, shuffle = True, random_state = 42)
    
    weights = None
    if weight_samples :
        weights = compute_sample_weight('balanced', df_train["task_position"])

    # build and fit model
    model = KerasClassifier(model = model_builder(sequence_length, len(message_type_encoder.classes_)+1), epochs = epochs, batch_size = batch_size, verbose = 1)
    model.fit(x_train, y_train, sample_weight = weights)

    # evaluate on training data
    test_pred = model.predict(x_test)
    evaluate(test_pred, y_test, label_encoder.classes_.tolist())

    # evaluate on interleaved data
    valid_pred = model.predict(valid_feature_sequence)
    evaluate_interleaved(valid_pred, valid_labels, label_encoder.classes_.tolist())
    return model


In [7]:
bl_model = pipeline(df_r4, df_r4_interleaved, baseline_model, sequence_length = 50, epochs = 100, batch_size = 1000)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

Epoch 1/100
27/27 [==============================] - 6s 21ms/step - loss: 0.5334 - f1_score: 0.3626
Epoch 2/100
27/27 [==============================] - 0s 13ms/step - loss: 0.2446 - f1_score: 0.3235
Epoch 3/100
27/27 [==============================] - 0s 13ms/step - loss: 0.2179 - f1_score: 0.3235
Epoch 4/100
27/27 [==============================] - 0s 13ms/step - loss: 0.1692 - f1_score: 0.3235
Epoch 5/100
27/27 [==============================] - 0s 13ms/step - loss: 0.1091 - f1_score: 0.5586
Epoch 6/100
27/27 [==============================] - 0s 13ms/step - loss: 0.0614 - f1_score: 0.7391
Epoch 7/100
27/27 [==============================] - 0s 13ms/step - loss: 0.0446 - f1_score: 0.8857
Epoch 8/100
27/27 [==============================] - 0s 13ms/step - loss: 0.0392 - f1_score: 0.9156
Epoch 9/100
27/27 [==============================] - 0s 13ms/step - loss: 0.0367 - f1_score: 0.9167
Epoch 10/100
27/27 [==============================] - 0s 13ms/step - loss: 0.0351 - f1_score: 0.9183

In [42]:
bl_model_weighted = pipeline(df_r4, df_r4_interleaved, baseline_model, sequence_length = 50, epochs = 100, batch_size = 1000, weight_samples=True)

NameError: name 'pipeline' is not defined

In [8]:
bl_model = pipeline(df_r1, df_r1_interleaved, baseline_model, sequence_length = 50, epochs = 100, batch_size = 1000)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

Epoch 1/100
1285/1285 [==============================] - 18s 12ms/step - loss: 0.0149 - f1_score: 0.3330
Epoch 2/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0044 - f1_score: 0.3510
Epoch 3/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0028 - f1_score: 0.5539
Epoch 4/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0020 - f1_score: 0.7259
Epoch 5/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0017 - f1_score: 0.7772
Epoch 6/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0014 - f1_score: 0.8165
Epoch 7/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0012 - f1_score: 0.8391
Epoch 8/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0011 - f1_score: 0.8562
Epoch 9/100
1285/1285 [==============================] - 15s 12ms/step - loss: 0.0011 - f1_score: 0.8631
Epoch 10/100
1285/1285 [==============================]

In [9]:
bl_r1_model = pipeline(df_r1, df_r1_interleaved, baseline_model, sequence_length = 25, epochs = 50, batch_size = 500)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

Epoch 1/50
2570/2570 [==============================] - 18s 6ms/step - loss: 0.0098 - f1_score: 0.3369
Epoch 2/50
2570/2570 [==============================] - 14s 6ms/step - loss: 0.0019 - f1_score: 0.7501
Epoch 3/50
2570/2570 [==============================] - 15s 6ms/step - loss: 0.0012 - f1_score: 0.8643
Epoch 4/50
2570/2570 [==============================] - 15s 6ms/step - loss: 0.0011 - f1_score: 0.8832
Epoch 5/50
2570/2570 [==============================] - 15s 6ms/step - loss: 9.7702e-04 - f1_score: 0.8855
Epoch 6/50
2570/2570 [==============================] - 15s 6ms/step - loss: 8.4974e-04 - f1_score: 0.9042
Epoch 7/50
2570/2570 [==============================] - 15s 6ms/step - loss: 7.8418e-04 - f1_score: 0.9125
Epoch 8/50
2570/2570 [==============================] - 15s 6ms/step - loss: 7.4225e-04 - f1_score: 0.9170
Epoch 9/50
2570/2570 [==============================] - 15s 6ms/step - loss: 6.7870e-04 - f1_score: 0.9219
Epoch 10/50
2570/2570 [==============================

In [10]:
bl_r1_model = pipeline(df_r1, df_r1_interleaved, baseline_model, sequence_length = 15, epochs = 50, batch_size = 500)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

Epoch 1/50
2570/2570 [==============================] - 15s 5ms/step - loss: 0.0102 - f1_score: 0.3331
Epoch 2/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0024 - f1_score: 0.6512
Epoch 3/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0017 - f1_score: 0.7756
Epoch 4/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0015 - f1_score: 0.8023
Epoch 5/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0014 - f1_score: 0.8161
Epoch 6/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0013 - f1_score: 0.8374
Epoch 7/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0012 - f1_score: 0.8409
Epoch 8/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0012 - f1_score: 0.8437
Epoch 9/50
2570/2570 [==============================] - 12s 5ms/step - loss: 0.0011 - f1_score: 0.8470
Epoch 10/50
2570/2570 [==============================] - 12s 5ms/step - l

In [ ]:
# Experiment with dropout

# Experiment with forward facing sliding window


In [ ]:
df_train = df_r1
df_valid = df_r1_interleaved
sequence_length = 50
epochs = 100
batch_size = 1000
model_builder = baseline_model

# relabel task position for interleaved data
df_valid["task_position"] = df_valid.apply(lambda row: "position_start" if row["real_single_activity_action"] == "Activity Start" else ("position_end" if row["real_single_activity_action"] == "Activity End" else "position_between"), axis = 1)

# mark event positions in training data
mark_event_positions(df_train)

# encode message type for both training and validation data
message_type_encoder = LabelEncoder().fit(df_train["MessageType"].values)
train_features = message_type_encoder.transform(df_train["MessageType"]) +1
valid_features = message_type_encoder.transform(df_valid["MessageType"]) +1 

# generate feature sequences 
train_feature_sequence = sliding_window(train_features,sequence_length)
valid_feature_sequence = sliding_window(valid_features, sequence_length)

# one hot encode labels
label_encoder = LabelEncoder().fit(df_train["task_position"])
train_labels = to_categorical(label_encoder.transform(df_train["task_position"]))
valid_labels = to_categorical(label_encoder.transform(df_valid["task_position"]))

# train test split
x_train, x_test, y_train, y_test = train_test_split(train_feature_sequence, train_labels, test_size = 0.2, shuffle = True, random_state = 42)


# build and fit model
model = KerasClassifier(model = model_builder(sequence_length, len(message_type_encoder.classes_)+1), epochs = epochs, batch_size = batch_size, verbose = 1)
model.fit(x_train, y_train, sample_weight = None)

# evaluate on training data
test_pred = model.predict(x_test)
evaluate(test_pred, y_test, label_encoder.classes_tolist())

# evaluate on interleaved data
valid_pred = model.predict(valid_feature_sequence)
evaluate_interleaved(valid_pred, valid_labels, label_encoder.classes_.tolist())




In [ ]:
df_train = df_r1
df_valid = df_r1_interleaved
sequence_length = 50
epochs = 100
batch_size = 1000
model_builder = baseline_model

# relabel task position for interleaved data
df_valid["task_position"] = df_valid.apply(lambda row: "position_start" if row["real_single_activity_action"] == "Activity Start" else ("position_end" if row["real_single_activity_action"] == "Activity End" else "position_between"), axis = 1)

# mark event positions in training data
mark_event_positions(df_train)

# encode message type for both training and validation data
message_type_encoder = LabelEncoder().fit(df_train["MessageType"].values)
train_features = message_type_encoder.transform(df_train["MessageType"]) +1
valid_features = message_type_encoder.transform(df_valid["MessageType"]) +1 

# generate feature sequences 
train_feature_sequence = sliding_window(train_features,sequence_length)
valid_feature_sequence = sliding_window(valid_features, sequence_length)

# one hot encode labels
label_encoder = LabelEncoder().fit(df_train["task_position"])
train_labels = to_categorical(label_encoder.transform(df_train["task_position"]))
valid_labels = to_categorical(label_encoder.transform(df_valid["task_position"]))

# train test split
x_train, x_test, y_train, y_test = train_test_split(train_feature_sequence, train_labels, test_size = 0.2, shuffle = True, random_state = 42)


# build and fit model
model = KerasClassifier(model = model_builder(sequence_length, len(message_type_encoder.classes_)+1), epochs = epochs, batch_size = batch_size, verbose = 1)
model.fit(x_train, y_train, sample_weight = None)

# evaluate on training data
test_pred = model.predict(x_test)
evaluate(test_pred, y_test, label_encoder.classes_tolist())

# evaluate on interleaved data
valid_pred = model.predict(valid_feature_sequence)
evaluate_interleaved(valid_pred, valid_labels, label_encoder.classes_.tolist())




In [ ]:
test_pred

In [ ]:
y_test

In [ ]:
evaluate(test_pred, y_test, label_encoder.classes_.tolist())

In [ ]:
# TODO index of for backward is not greedy - reverse! 
import random
def evaluate_interleaved(pred, y, labels):
    start_window = 27
    end_window = 51
    
    
    # transform input to integer labels
    pred_int = np.argmax(pred, axis = 1).tolist()
    y_int = np.argmax(y, axis = 1).tolist()
    
    in_between = labels.index("position_between")
    end = labels.index("position_end")
    start = labels.index("position_start")
    
    # For predicted R1 start events, the R4 label should be in a forward facing window of size 27
    # For predicted R1 end events the R4 label should be in a backward facing window of size 51
    # Since we introduce a window for the R4 labels, the original labels should be changed to "in between". 
    
    # get classification results for Start Events 
    
    # Idea: Reorder the labels greedy based on the window sizes! 
    # For start events in pred: Find swap candidate in y_int sublist index + window size, swap "forward"
    # For end events in results: Find swap candidate in pred index + size, swap "backward"
    # Todo Investigate: Only allow swaps with "Between elements"
        
    for idx in range(len(pred)):

        if(pred_int[idx] == start):
            try:# start event in predictions found -> check for start event in labels with forward facing window "forward swapping"            
                res_idx = y_int.index(start, idx,idx+start_window)
                if(y_int[idx] == in_between):
                    y_int[idx], y_int[res_idx] = y_int[res_idx], y_int[idx]
            except:
                pass

        bw_idx = len(pred)-idx-1
        if(pred_int[bw_idx]==end):
            try: 
                res_idx = y_int.index(end, max(bw_idx-end_window,0),bw_idx)
                if y_int[bw_idx] == in_between :
                    y_int[bw_idx], y_int[res_idx] = y_int[res_idx], y_int[bw_idx]
            except: 
                pass
    
    report = classification_report(y_int, pred_int, target_names = labels)
    confusion_matrix = multilabel_confusion_matrix(y_int, pred_int)
    print(report)
    print(confusion_matrix)
       
pred_random = startSample() + betweenSample(50) + endSample()
labels_random = betweenSample(10) + startSample(1) + betweenSample(30) + endSample() + betweenSample(10)


In [ ]:
def mark_event_positions(df):
    # Mark start event of each BusinessActivity Instance
    df["activityStart"] = df.groupby(["BusinessActivity","InstanceNumber",]).cumcount()==0
    # Mark end event of each Business Activity Instance
    df["activityEnd"] = df.groupby(["BusinessActivity","InstanceNumber",]).cumcount(ascending=False)==0
    # Merge start and end columns to form labels
    df["task_position"] = df.apply(lambda row: "position_start" if row["activityStart"] else ("position_end" if row["activityEnd"] else 'position_between'), axis=1)

    df = df.drop(["activityStart",'activityEnd'], axis=1)
    
mark_event_positions(df_r4)
mark_event_positions(df_r1)

In [ ]:
# Label Encode the Message Type
messageTypeEncoder = LabelEncoder()
messageTypeEncoder.fit(df["MessageType"].values)
# Shift the labels by 1 to exclude 0 as label (we use it for padding starting sequences)
# Train Data
labels = messageTypeEncoder.transform(df["MessageType"]) + 1

# Interleaved data
labels_interleaved = messageTypeEncoder.transform(df_interleaved["MessageType"]) + 1


In [ ]:
# One hot encode the Labels
encoder = LabelEncoder()
y = encoder.fit_transform(df["task_position"])
y_interleaved = encoder.transform(df_interleaved["task_position"])
dummy_y = to_categorical(y)
dummy_y_interleaved = to_categorical(y_interleaved)

In [ ]:
def sliding_window(observation, sequence_length = 50): 
    sequences = sliding_window_view(observation, sequence_length)
    
    # pad first n-1 elements
    pad_top = [[0] * (sequence_length - 1 - i) + sequences[0, 0:i + 1].tolist() for i in range(sequence_length - 1)]
    
    res = np.insert(sequences,0, pad_top, axis=0)
    #res = np.asarray(X).astype(np.float32) 
    #res = res.reshape(res.shape[0], res.shape[1], 1)
    return res

In [ ]:
from keras.src.layers import LSTM
from keras.layers import Bidirectional
from keras import Input
from keras.metrics import F1Score

def baseline_model(sequence_length):
    embedding_dim = 150

    vocab_size = len(messageTypeEncoder.classes_) + 1
    model = Sequential([
        Input(shape=(sequence_length,)),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        LSTM(units=64),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[F1Score()])
    return model


In [ ]:
def evaluate(model, x_test, y_test):
    pred = model.predict(x_test)
    report = classification_report(y_test, pred, target_names = encoder.classes_)
    confusion_matrix = multilabel_confusion_matrix(y_test, pred)
    print(report)
    print(confusion_matrix)
    return report, confusion_matrix
    

In [ ]:
def fit_and_evaluate(features, labels, model, sequence_length = 50, epochs = 100, batch_size = 500):
    x = sliding_window(features,sequence_length)
    x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size = 0.2, shuffle = True, random_state = 42)
    model = KerasClassifier(model = model(sequence_length), epochs = epochs, batch_size = batch_size, verbose = 1)
    model.fit(x_train, y_train, sample_weight = None)
    evaluate(model, x_test, y_test)
    return model

In [ ]:
bl_50_model = fit_and_evaluate(labels,dummy_y,baseline_model, 50, 50, 250)

In [ ]:
bl_20_model = fit_and_evaluate(labels,dummy_y,baseline_model, 20, 50, 250)

In [ ]:
bl_15_model = fit_and_evaluate(labels,dummy_y,baseline_model, 15, 100, 250)

In [ ]:
bl_10_model = fit_and_evaluate(labels,dummy_y,baseline_model, 10, 100, 250)

## Evaluate on inverleaved data

In [ ]:
x_50_interleaved = sliding_window(labels_interleaved,50)
res_50 = evaluate(bl_50_model, x_50_interleaved, dummy_y_interleaved)

In [ ]:
x_20_interleaved = sliding_window(labels_interleaved,20)
res_20 = evaluate(bl_20_model, x_20_interleaved, dummy_y_interleaved)

In [ ]:
len(dummy_y_interleaved)